In [1]:
GOOGLE_API_CREDENTIAL = "/Users/tyoyo/.secrets/tyoyo-project-1d00c168b221.json"

def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    import google
    from google.cloud import speech
    import io

    credentials = (
        google.oauth2.service_account.Credentials.from_service_account_file(
            GOOGLE_API_CREDENTIAL
        )
    )
    client = speech.SpeechClient(credentials=credentials)

    with io.open(speech_file, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code="ja-JP",
    )

    response = client.recognize(config=config, audio=audio)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(u"Transcript: {}".format(result.alternatives[0].transcript))
    return response


In [2]:
response = transcribe_file('data/1911F2002.wav')

ValueError: Unknown field for RecognitionConfig: enable_speaker_diarization

In [13]:
type(response)

google.cloud.speech_v1.types.cloud_speech.RecognizeResponse

In [50]:
def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    from google.cloud import speech
    import google
    credentials = (
        google.oauth2.service_account.Credentials.from_service_account_file(
            GOOGLE_API_CREDENTIAL
        )
    )
    client = speech.SpeechClient(credentials=credentials)

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
#         encoding=speech.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=48000,
        language_code="ja-JP",
#         enable_speaker_diarization=True,
#         diarization_speaker_count=2,
    )

    operation = client.long_running_recognize(config=config, audio=audio)


    print("Waiting for operation to complete...")
    response = operation.result(timeout=360)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(u"Transcript: {}".format(result.alternatives[0].transcript))
        print("Confidence: {}".format(result.alternatives[0].confidence))
    return response

In [49]:
response = transcribe_gcs('gs://tyoyo/onlinesum/1911F2002.wav')

ValueError: Unknown field for RecognitionConfig: enable_speaker_diarization

In [16]:
transcripts = [result.alternatives[0].transcript for result in response.results]

In [20]:
with open('data/transcripts/1911F2002.txt', 'w') as f:
    f.write('\n'.join(transcripts))

In [45]:
import jsonlines

with jsonlines.open('data/stt_transcripts/1911F2002.jsonl', 'w') as writer:
    for result in response.results:
        writer.write({'transcript': result.alternatives[0].transcript, 'confidence': result.alternatives[0].confidence})

In [36]:
response.results[0].

alternatives {
  transcript: "\343\201\223\343\202\214\343\201\213\343\202\211\345\220\221\343\201\213\343\201\204\343\201\276\343\201\231"
  confidence: 0.7493587136268616
}